### Supplementary Figure Generator

# Spatial-Diffraction Induced Phasor Averaging

This script generates a composite 5-panel supplementary figure to illustrate how the
microscope's Point Spread Function (PSF) causes spatial averaging of fluorescence lifetimes
at boundaries between different species. This averaging results in a linear combination
in phasor space.

The script performs the following steps:
1. Define simulation parameters (modulation frequency, lifetimes).
2. Generate 5 individual panels (a-e) using Matplotlib:
    - (a) Schematic of two spatial domains and the PSF.
    - (b) Time-domain decay curves (pure vs. mixed).
    - (c) Phasor plot showing the linear mixing rule.
    - (d) 2D heatmap of apparent lifetime across an interface.
    - (e) Effect of PSF size on the measured phasor position.
3. Assemble these panels into a single vertical composite image using the Pillow (PIL) library.


In [1]:
## Import libraries
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont
import io, os, math

## Define helper functions

In [2]:
## Mono-exponential phasor

def phasor_mono(omega, tau):
    """
    Calculates the phasor coordinates (g, s) for a single-exponential decay.

    Args:
        omega (float): Angular modulation frequency (rad/ns).
        tau (float): Fluorescence lifetime (ns).

    Returns:
        tuple: (g, s) coordinates.
    """
    g = 1.0 / (1.0 + (omega * tau)**2)
    s = (omega * tau) / (1.0 + (omega * tau)**2)
    return g, s

In [3]:
## Global parameters for the demo
# Modulation frequency setup
f_MHz = 80.0
omega = 2.0 * np.pi * (f_MHz / 1000.0)  # Convert MHz to rad/ns (1 ns^-1 = 1 GHz)

# Define two distinct lifetime species
tau_A = 1.0   # ns (Short lifetime species)
tau_B = 4.0   # ns (Long lifetime species)

# Calculate their theoretical phasor positions
gA, sA = phasor_mono(omega, tau_A)
gB, sB = phasor_mono(omega, tau_B)

# Define weights for a hypothetical pixel sitting on the boundary
# A weight of 0.5 implies the PSF collects equal photons from Domain A and Domain B
wA = 0.5
wB = 1.0 - wA

# Calculate the mixed phasor coordinates (Linear Combination Rule)
gM = wA * gA + wB * gB
sM = wA * sA + wB * sB

In [4]:

## Panel (a): Two domains + Gaussian PSF overlap schematic

def make_panel_a(path):
    """
    Generates Panel A: A spatial schematic showing two distinct lifetime domains
    and a Gaussian PSF centered at the interface.
    """
    W, H = 900, 600
    fig = plt.figure(figsize=(W/100, H/100), dpi=300)
    ax = plt.gca()

    # Setup coordinate system
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.set_aspect('equal')

    # Draw Domain A (Left, Blue-ish) and Domain B (Right, Red-ish)
    ax.add_patch(plt.Rectangle((0,0), 0.5, 1.0, facecolor=(0.8,0.92,1.0), edgecolor='none'))
    ax.add_patch(plt.Rectangle((0.5,0), 0.5, 1.0, facecolor=(1.0,0.9,0.9), edgecolor='none'))

    # Draw Boundary line
    ax.plot([0.5, 0.5], [0, 1], lw=2, color='k', alpha=0.3)

    # Generate and draw Gaussian PSF centered at boundary pixel r0 (0.5, 0.5)
    x = np.linspace(0, 1, 300)
    y = np.linspace(0, 1, 300)
    X, Y = np.meshgrid(x, y)
    x0, y0 = 0.5, 0.5
    sigma = 0.10

    # 2D Gaussian formula
    PSF = np.exp(-((X - x0)**2 + (Y - y0)**2) / (2 * sigma**2))
    PSF = PSF / PSF.max() # Normalize for visualization

    # Overlay PSF as a semi-transparent heatmap
    ax.imshow(PSF, extent=(0, 1, 0, 1), origin='lower', cmap='gray', alpha=0.45)

    # Annotations
    ax.text(0.25, 0.9, "Domain A (τ_A = %.1f ns)" % tau_A, ha='center', va='center', fontsize=16)
    ax.text(0.75, 0.9, "Domain B (τ_B = %.1f ns)" % tau_B, ha='center', va='center', fontsize=16)
    ax.text(x0, y0 - 0.16, "PSF at pixel r₀", ha='center', fontsize=14)
    ax.scatter([x0], [y0], s=120, c='k') # Mark pixel center

    # Formatting
    ax.set_xticks([]); ax.set_yticks([])
    ax.set_title("(a) Two spatial lifetime domains and the diffraction-limited PSF", fontsize=18, pad=12)

    fig.tight_layout()
    fig.savefig(path, dpi=300, bbox_inches='tight')
    plt.close(fig)

In [5]:
## Panel (b): Time-Domain Decays
def make_panel_b(path):
    """
    Generates Panel B: Illustrates the time-domain fluorescence decays.
    Shows the pure decays (I_A, I_B) and the measured mixed decay (I_meas)
    resulting from spatial averaging.
    """
    fig = plt.figure(figsize=(9, 5), dpi=300)
    ax = plt.gca()

    t = np.linspace(0, 12.5, 600)  # Time axis in ns
    A_A, A_B = 1.0, 1.0 # Amplitudes

    # Generate decay curves
    I_A = A_A * np.exp(-t / tau_A)
    I_B = A_B * np.exp(-t / tau_B)

    # Mixed decay is the weighted sum of intensities
    I_meas = wA * I_A + wB * I_B

    # Plotting
    ax.plot(t, I_A, lw=2, label="I_A(t) = A_A e^{-t/τ_A}")
    ax.plot(t, I_B, lw=2, label="I_B(t) = A_B e^{-t/τ_B}")
    ax.plot(t, I_meas, lw=3, linestyle='--', label="I_meas(t) = w_A I_A + w_B I_B")

    ax.set_xlabel("Time (ns)")
    ax.set_ylabel("Intensity (a.u.)")
    ax.set_title("(b) True local decays vs. PSF-weighted mixed decay at the interface", pad=10)
    ax.legend(loc='upper right', frameon=False)

    fig.tight_layout()
    fig.savefig(path, dpi=300, bbox_inches='tight')
    plt.close(fig)

In [6]:

## Panel (c): Phasor Plot Representation
def make_panel_c(path):
    """
    Generates Panel C: The Phasor Plot.
    Demonstrates that the phasor of the mixed signal falls on the line connecting
    the phasors of the pure species A and B.
    """
    fig = plt.figure(figsize=(7, 7), dpi=300)
    ax = plt.gca()

    # Draw Universal Semicircle (Theoretical limit for single exponentials)
    theta = np.linspace(0, np.pi, 400)
    semicircle_g = 0.5 * (1 + np.cos(theta))
    semicircle_s = 0.5 * np.sin(theta)
    ax.plot(semicircle_g, semicircle_s, lw=2)

    # Plot Phasor Points
    ax.scatter([gA], [sA], s=80) # Pure A
    ax.scatter([gB], [sB], s=80) # Pure B
    ax.scatter([gM], [sM], s=90, marker='D') # Mixed M

    # Draw connecting line (The "Mixing Line")
    ax.plot([gA, gB], [sA, sB], linestyle='--', lw=1.5)

    # Labels and formatting
    ax.text(gA, sA + 0.03, "G(τ_A)", ha='center')
    ax.text(gB, sB + 0.03, "G(τ_B)", ha='center')
    ax.text(gM, sM - 0.05, "G_meas", ha='center')

    ax.set_xlabel("g (in-phase)")
    ax.set_ylabel("s (quadrature)")
    ax.set_title("(c) Phasor averaging: G_meas = w_A G(τ_A) + w_B G(τ_B)", pad=10)
    ax.set_xlim(0, 1.02); ax.set_ylim(0, 0.55)
    ax.set_aspect('equal')
    ax.grid(alpha=0.2)

    fig.tight_layout()
    fig.savefig(path, dpi=300, bbox_inches='tight')
    plt.close(fig)

In [7]:

##Panel (d): 2D Apparent Lifetime Map
def make_panel_d(path):
    """
    Generates Panel D: A simulated 2D scan across the interface.
    Calculates the 'apparent' phase lifetime for every pixel by convolving
    the step-function interface with the Gaussian PSF.
    """
    # Create a 2D spatial grid
    W, H = 240, 160
    x = np.linspace(-1.2, 1.2, W)
    y = np.linspace(-0.8, 0.8, H)
    X, Y = np.meshgrid(x, y)

    sigma = 0.15 # PSF size for this panel

    # Define weighting function based on error function (integral of Gaussian)
    # This represents how much of the "Left" domain is inside the PSF centered at x0
    def weight_left(x0):
        # 0.5 * (1 + erf( ... )) gives the CDF of the Gaussian
        return 0.5 * (1.0 + math.erf((-x0) / (math.sqrt(2) * sigma)))

    # Initialize arrays for apparent phasors
    g_app = np.zeros_like(X)
    s_app = np.zeros_like(X)

    # Compute mixed phasor for each column (invariant in y direction)
    for ix, x0 in enumerate(x):
        wL = weight_left(x0) # Weight of Left Domain (A)
        wR = 1.0 - wL        # Weight of Right Domain (B)

        # Linear combination of phasors
        g_mix = wL * gA + wR * gB
        s_mix = wL * sA + wR * sB

        g_app[:, ix] = g_mix
        s_app[:, ix] = s_mix

    # Convert mixed phasor back to 'Apparent Lifetime' via phase angle
    # tau_phi = tan(phi) / omega = (s/g) / omega
    tau_phi = np.arctan2(s_app, g_app) / omega

    # Plot Heatmap
    fig = plt.figure(figsize=(8, 4.6), dpi=300)
    ax = plt.gca()
    im = ax.imshow(tau_phi, extent=(x.min(), x.max(), y.min(), y.max()),
                   origin='lower', aspect='auto')

    cbar = plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    cbar.set_label("Apparent phase lifetime τ_phi (ns)")

    ax.set_title("(d) Apparent lifetime map across the A|B interface due to spatial averaging", pad=8)
    ax.set_xlabel("x (arb. units)")
    ax.set_ylabel("y (arb. units)")

    fig.tight_layout()
    fig.savefig(path, dpi=300, bbox_inches='tight')
    plt.close(fig)

In [8]:

## Panel (e): Effect of PSF Size
def make_panel_e(path):
    """
    Generates Panel E: Illustrates how changing the PSF size affects the measurement.
    A larger PSF averages more area, potentially shifting the weights closer to 50/50
    if the domains are large, or changing the ratio if the domains are unequal.
    """
    fig = plt.figure(figsize=(7, 7), dpi=300)
    ax = plt.gca()

    # Universal semicircle
    theta = np.linspace(0, np.pi, 400)
    semicircle_g = 0.5 * (1 + np.cos(theta))
    semicircle_s = 0.5 * np.sin(theta)
    ax.plot(semicircle_g, semicircle_s, lw=2)

    # Simulate two different PSF widths resulting in different weights at the same center pixel
    # Small PSF: localized, sees mostly domain A (wA ~ 0.65)
    # Large PSF: blurred, sees more domain B mix (wA ~ 0.50)
    wA_small = 0.65
    wA_large = 0.50

    # Calculate mixed phasors for both cases
    g_small = wA_small * gA + (1 - wA_small) * gB
    s_small = wA_small * sA + (1 - wA_small) * sB

    g_large = wA_large * gA + (1 - wA_large) * gB
    s_large = wA_large * sA + (1 - wA_large) * sB

    # Plot
    ax.plot([gA, gB], [sA, sB], linestyle='--', lw=1.5) # Mixing line
    ax.scatter([gA], [sA], s=80)
    ax.scatter([gB], [sB], s=80)
    ax.scatter([g_small], [s_small], s=100, marker='o') # Small PSF point
    ax.scatter([g_large], [s_large], s=100, marker='D') # Large PSF point

    # Annotations
    ax.text(gA, sA + 0.04, "$(g_A,s_A)$($\u03c4_A$)", ha='center')
    ax.text(gB, sB + 0.04, "$(g_B,s_B)$($\u03c4_B$)", ha='center')
    ax.text(g_small - 0.05, s_small + 0.03, "small PSF", ha='left', va='center')
    ax.text(g_large - 0.05, s_large - 0.03, "large PSF", ha='left', va='center')

    ax.set_xlabel("$G$")
    ax.set_ylabel("$S$")
    ax.set_title("Changing PSF size shifts the measured phasor at interfaces", pad=10)
    ax.set_xlim(0, 1.02); ax.set_ylim(0, 0.55)
    ax.set_aspect('equal')
    ax.grid(alpha=0.2)

    fig.tight_layout()
    fig.savefig(path, dpi=300, bbox_inches='tight')
    plt.close(fig)

In [9]:
## Generate panels and assemble composite
out_dir = "."
panel_paths = []

# Loop through panel generation functions
for name, maker in [
    ("panel_a.png", make_panel_a),
    ("panel_b.png", make_panel_b),
    ("panel_c.png", make_panel_c),
    ("panel_d.png", make_panel_d),
    ("panel_e.png", make_panel_e),
]:
    p = os.path.join(out_dir, name)
    maker(p)
    panel_paths.append(p)

# Open generated images with Pillow
imgs = [Image.open(p).convert("RGB") for p in panel_paths]

# Resize all images to match the width of the widest panel
max_w = max(im.width for im in imgs)
resized = []
for im in imgs:
    if im.width != max_w:
        # Calculate new height to maintain aspect ratio
        new_h = int(im.height * (max_w / im.width))
        # Use high-quality resampling (LANCZOS)
        im = im.resize((max_w, new_h), Image.Resampling.LANCZOS)
    resized.append(im)

# Create a blank composite image (White background)
gap = 18 # Vertical gap between panels
total_h = sum(im.height for im in resized) + gap * (len(resized) - 1)
composite = Image.new("RGB", (max_w, total_h), (255, 255, 255))

y = 0
labels = ['(a)', '(b)', '(c)', '(d)', '(e)']
draw = ImageDraw.Draw(composite)

# Paste images and add labels
for i, im in enumerate(resized):
    composite.paste(im, (0, y))

    # Draw a small white box behind the label for legibility
    draw.rectangle([8, y + 8, 58, y + 40], fill=(255, 255, 255))

    # Draw text label (e.g., "(a)")
    draw.text((12, y + 12), labels[i], fill=(0, 0, 0))

    y += im.height + gap

# Save final output
composite_path = os.path.join(out_dir, "SupplementaryFigure_Phasor_SpatialDiffraction.png")
composite.save(composite_path, "PNG")

print("Saved Composite:", composite_path)
for p in panel_paths:
    print("Saved Panel:", p)

Saved Composite: ./SupplementaryFigure_Phasor_SpatialDiffraction.png
Saved Panel: ./panel_a.png
Saved Panel: ./panel_b.png
Saved Panel: ./panel_c.png
Saved Panel: ./panel_d.png
Saved Panel: ./panel_e.png
